<a href="https://colab.research.google.com/github/subarudad/gh-collab/blob/main/github_code_understanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# github repo instruction text to code

In [ ]:
!pip install langchain openai chromadb

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# !pip install deeplake

# https://www.activeloop.ai/resources/lang-chain-gpt-4-for-code-understanding-twitter-algorithm/

# https://python.langchain.com/docs/use_cases/code/code-analysis-deeplake

from getpass import getpass

os.environ["ACTIVELOOP_TOKEN"] = getpass.getpass("Activeloop Token:")

# os.environ['ACTIVELOOP_TOKEN']='ACTIVELOOP_TOKEN'


In [ ]:
from langchain import LLMChain, OpenAI, Cohere, HuggingFaceHub


In [ ]:
import os
from langchain.document_loaders import TextLoader

root_dir = './the-algorithm'

# scrape any files from directory

docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try:
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e:
            pass


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


TEXT_SPLITTER_CHUNK_PARAMS = {
    "chunk_size": 2000,
    "chunk_overlap": 0,
    "length_function": len,
}

text_splitter = RecursiveCharacterTextSplitter(**TEXT_SPLITTER_CHUNK_PARAMS)
texts = text_splitter.split_documents(docs)


In [ ]:

# https://python.langchain.com/docs/modules/data_connection/vectorstores/integrations/chroma

In [ ]:


from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma


embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

text_vdb = Chroma.from_texts(texts, embeddings, metadatas=metadatas, dataset_path=dataset_path)



In [ ]:
# from langchain.vectorstores import DeepLake

# db = DeepLake.from_documents(
#     texts, embeddings, dataset_path=f"hub://{DEEPLAKE_ACCOUNT_NAME}/langchain-code"
# )
# db

# replace with your username from app.activeloop.ai
# db = DeepLake(dataset_path=f"hub://{username}/twitter-algorithm", embedding_function=embeddings)
# db.add_documents(texts)


In [ ]:
prompt = 'What are the first programs he tried writing?'
query_docs = text_vdb.similarity_search(query = prompt)
print(query_docs[0].page_content)



In [ ]:

retriever = text_vdb.as_retriever()
retriever.search_kwargs["exec_option"] = "compute_engine"
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['fetch_k'] = 100
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 10

# retriever = db.as_retriever(search_type="mmr")

retriever.get_relevant_documents(query)[0]


In [ ]:
from langchain.llms import OpenAI
from langchain.llms import OpenAIChat

from langchain.chains import RetrievalQA

qna = RetrievalQA.from_chain_type(llm = OpenAIChat(model = 'gpt-3.5-turbo'),
                                  chain_type = 'stuff', retriever = retriever)

qna.run(prompt)


In [ ]:
questions = [
    "What is the class hierarchy?",
    # "What classes are derived from the Chain class?",
    # "What classes and functions in the ./langchain/utilities/ forlder are not covered by unit tests?",
    # "What one improvement do you propose in code in relation to the class herarchy for the Chain class?",
]
chat_history = []

for question in questions:
    result = qna({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")